In [1]:
import json

from collections import OrderedDict
from pprint import pprint

import httplib2
import numpy as np
import numpy.linalg
import numpy.random
import scipy.optimize

from apiclient.discovery import build
from oauth2client import tools
from oauth2client.client import OAuth2WebServerFlow
from oauth2client.file import Storage

In [2]:
route = {}
route['Morning'] = OrderedDict(
    {'Hill St': [25, 40, 30],
     'Hegeman Bridge Rd': [30, 50, 40],
     'Rt. 40': [45, 60, 50],
     'Sarles Ferry': [40, 50, 45],
     'CR-113': [45, 60, 50],
     'Stillwater Bridge Rd': [25, 40, 35],
     'Stillwater': [25, 40, 30],
     'Hill in Stillwater': [25, 40, 35],
     'Rt. 4': [35, 45, 40],
     'Mechanicville': [25, 40, 30],
     'Leaving Mechanicville': [25, 45, 35],
     '45 mph Rt. 4': [40, 55, 45],
     'Rt. 146 hill': [40, 60, 45],
     'Rt. 146 curve': [45, 60, 50],
     'Rt. 146 before light': [45, 60, 45],
     'Rt. 146 after light': [45, 55, 45],
     'Rt. 146 Halfmoon': [45, 55, 45],
     'Clifton Park': [40, 50, 45],
     'Hill by Vent Fitness': [40, 60, 45],
     '55 mph Rt. 146': [45, 60, 50],
     'Rt. 146 near Balltown': [40, 60, 45],
     'Balltown in Rexford': [40, 50, 45],
     'Balltown on bridge': [40, 50, 45],
     '45 mph Balltown': [40, 50, 45],
     '40 mph Balltown': [40, 50, 40],
     'River Rd': [35, 50, 40],
    })

In [3]:
route['Afternoon'] = OrderedDict(
    {'River Rd': [35, 50, 40],
     '40 mph Balltown': [40, 50, 40],
     '45 mph Balltown': [40, 50, 45],
     'Balltown on bridge': [40, 50, 45],
     'Balltown in Rexford': [40, 50, 45],
     'Rt. 146 near Balltown': [40, 60, 45],
     '55 mph Rt. 146': [45, 60, 50],
     'Hill by Vent Fitness': [40, 60, 45],
     'Clifton Park': [40, 50, 45],
     'Rt. 146 Halfmoon': [45, 55, 45],
     'Rt. 146 before light': [45, 60, 45],
     'Rt. 146 after light': [45, 60, 45],
     'Rt. 146 curve': [45, 60, 50],
     'Rt. 146 hill': [40, 60, 45],
     '45 mph Rt. 4': [40, 55, 45],
     'Entering Mechanicville': [30, 45, 35],
     'Mechanicville': [25, 40, 30],
     'Rt. 4': [35, 45, 40],
     'Hill in Stillwater': [25, 40, 35],
     'Stillwater': [25, 40, 30],
     'Stillwater Bridge Rd': [25, 40, 35],
     'CR-113': [45, 60, 50],
     'Sarles Ferry': [40, 50, 45],
     'Rt. 40': [45, 60, 50],
     'Hegeman Bridge Rd': [30, 50, 40],
     'Hill St': [25, 40, 30],
    })

In [4]:
def extract_initial_values(dset, route=route):
    return [bds[2] for bds in route[dset].values()]
x0 = {}
x0['Morning'] = extract_initial_values('Morning')
x0['Afternoon'] = extract_initial_values('Afternoon')

In [5]:
data = {}
data['Morning'] = []
data['Afternoon'] = []

In [6]:
lo = np.linalg.norm([bds[0] for bds in route['Morning'].values()])
hi = np.linalg.norm([bds[1] for bds in route['Morning'].values()])

In [165]:
ITER = None
def fun(x, dset):
    global ITER
    x = np.array(np.rint(x), dtype=np.int)
    y = np.random.normal(5 * (np.linalg.norm(x) - hi) / (lo - hi) + 20, 0.5)
    print(f'{dset} {ITER}: MPG for {x} = {y}')
    ITER += 1
    return 10000 / y

In [155]:
def opt(dset, method, route=route):
    global ITER
    ITER = 1
    return scipy.optimize.minimize(fun, x0[dset], args=(dset,), method=method,
                                   options={'eps': 5},
                                   bounds=list(x[:2] for x in route[dset].values()))

In [150]:
def evaluate(method):
    niters = 10000
    fun = np.empty(niters, dtype=float)
    nfev = np.empty(niters, dtype=int)
    success = np.empty(niters, dtype=bool)
    for i in range(niters):
        sol = opt('Morning', method)
        fun[i] = 10000 / sol.fun
        nfev[i] = sol.nfev
        success = sol.success
    print(f'Solution: mean = {np.mean(fun)}, stdev = {np.std(fun)}')
    print(f'F_evals: mean = {np.mean(nfev)}, stdev = {np.std(nfev)}')
    print(f'Failure rate = {np.count_nonzero(success) / niters * 100}')

In [160]:
evaluate('L-BFGS-B')

Solution: mean = 23.438422482447162, stdev = 0.6531470873548814
F_evals: mean = 361.3815, stdev = 119.89842349985258
Failure rate = 0.01


In [161]:
evaluate('COBYLA')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:524: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: eps
  


Solution: mean = 23.286910781688498, stdev = 0.4962340148845377
F_evals: mean = 206.2147, stdev = 11.369679147187927
Failure rate = 0.01


In [162]:
evaluate('TNC')

Solution: mean = 23.31180097050842, stdev = 0.5069282763944383
F_evals: mean = 41.9992, stdev = 27.3919951693921
Failure rate = 0.0


In [163]:
evaluate('SLSQP')

Solution: mean = 23.473051368442555, stdev = 0.6058101386851343
F_evals: mean = 104.7638, stdev = 44.36745214185733
Failure rate = 0.01


In [166]:
opt('Morning', 'SLSQP')

Morning 1: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.66970547726557
Morning 2: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.658391140873402
Morning 3: MPG for [35 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.436775389504607
Morning 4: MPG for [30 45 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.188640546565384
Morning 5: MPG for [30 40 55 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.518520780346766
Morning 6: MPG for [30 40 50 50 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.739295868859642
Morning 7: MPG for [30 40 50 45 55 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 24.24922721887492
Morning 8: MPG for [30 40 50 45 50 40 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.04911646742555
Morning 9: MPG for [30 40 50 45 50 

     fun: 434.01122750738705
     jac: array([-1.22619223, -3.3494148 ,  1.6054168 ,  3.0207272 ,  1.10076527,
       -2.2410541 ,  2.58536329, -0.78473428,  1.04093531,  0.54549458,
        1.55952235,  0.80226229, -3.60659705,  1.67560166,  0.36150711,
        4.55743735,  4.27370055,  3.14033745, -0.37131786, -0.40374781,
        0.45870128,  0.21234125,  2.01842082,  1.66362676, -3.68732596,
       -0.87727926])
 message: 'Optimization terminated successfully.'
    nfev: 73
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 29.99985805,  39.99969591,  49.99991072,  44.99939323,
        50.00036576,  34.99960319,  29.99983021,  34.99974817,
        39.99974694,  29.99987906,  35.00017357,  44.99937979,
        44.9997948 ,  49.99990811,  45.00014652,  45.0000732 ,
        45.        ,  44.99988944,  44.99998499,  49.99972397,
        44.99972109,  44.99915811,  44.99911211,  45.00050787,
        40.00000001,  40.00026295])

In [167]:
opt('Morning', 'TNC')

Morning 1: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.786331693406073
Morning 2: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.735052176687205
Morning 3: MPG for [35 40 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.002670200676338
Morning 4: MPG for [30 45 50 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.70808822718245
Morning 5: MPG for [30 40 55 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.95550150809328
Morning 6: MPG for [30 40 50 50 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.24738532035466
Morning 7: MPG for [30 40 50 45 55 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.755378927820768
Morning 8: MPG for [30 40 50 45 50 40 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.23342344937548
Morning 9: MPG for [30 40 50 45 50 3

Morning 117: MPG for [30 40 55 45 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.34511854965311
Morning 118: MPG for [30 40 50 50 50 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.224822262297952
Morning 119: MPG for [30 40 50 45 55 35 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.542590054688926
Morning 120: MPG for [30 40 50 45 50 40 30 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.639298673754514
Morning 121: MPG for [30 40 50 45 50 35 35 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.51142781680476
Morning 122: MPG for [30 40 50 45 50 35 30 40 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.452715444033377
Morning 123: MPG for [30 40 50 45 50 35 30 35 45 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.28026224078692
Morning 124: MPG for [30 40 50 45 50 35 30 35 40 35 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.982632212295087
Morning 125: MPG fo

Morning 1297: MPG for [30 40 50 45 50 35 35 35 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 24.357809656941903
Morning 1298: MPG for [30 40 50 45 50 35 30 40 40 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.419645578404506
Morning 1299: MPG for [30 40 50 45 50 35 30 35 45 30 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.090411961235453
Morning 1300: MPG for [30 40 50 45 50 35 30 35 40 35 35 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 23.191535333966215
Morning 1301: MPG for [30 40 50 45 50 35 30 35 40 30 40 45 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.700052641807403
Morning 1302: MPG for [30 40 50 45 50 35 30 35 40 30 35 50 45 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 24.047904345724287
Morning 1303: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 50 50 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.62466672559822
Morning 1304: MPG for [30 40 50 45 50 35 30 35 40 30 35 45 45 55 45 45 45 45 45 50 45 45 45 45 40
 40] = 22.66111687301874
Morning 13

     fun: 426.76611716919035
     jac: array([ 1.82907008,  1.97904888,  4.98048119,  1.79148106,  2.01519009,
        1.28334552,  2.51537099,  1.62699584,  2.51675772,  0.58352194,
        0.78346752, -0.1772667 ,  3.4537794 ,  0.22032191,  3.59568559,
        4.95265543,  1.44496987,  0.73636687,  3.57327734,  0.87238166,
        5.70089239,  4.61051768,  1.64496924, -0.82414773,  4.17391333,
        4.32395295])
 message: 'Linear search failed'
    nfev: 104
     nit: 3
  status: 4
 success: False
       x: array([ 29.99514452,  40.0102824 ,  50.00136482,  45.00068812,
        50.001893  ,  34.98835344,  29.99966086,  35.00106229,
        40.00720167,  30.00755438,  34.99870594,  44.99847605,
        45.00026124,  49.99495728,  45.        ,  45.04762328,
        45.        ,  45.00387262,  44.9708662 ,  50.01590012,
        44.9954167 ,  45.00346232,  45.00396976,  45.00784225,
        40.00089042,  39.998258  ])